# Scraping mentions of ... on Twitter

### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import snscrape.modules.twitter as sntwitter
import itertools

In [3]:
%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('latimes')

In [5]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", None)

In [6]:
# https://medium.com/better-programming/how-to-scrape-tweets-with-snscrape-90124ed006af

In [7]:
# https://github.com/JustAnotherArchivist/snscrape

### Define search term and search dates

In [8]:
search = '"fuckshit"'

In [9]:
begin = "2021-01-01"

In [10]:
today = pd.to_datetime("today")

In [11]:
location = "near:Los Angeles"

In [12]:
radius = "50"

### Scrape using snscrape via command line

In [13]:
# arguments
# --progress (to see results in 100 tweet intervals)
# --max-results 500 (limit collection to last 500 tweets)
# --with-entity (include the entity — e.g. user, channel — as the first output item)

In [14]:
# !snscrape --jsonl --max-results 10 --since '{begin}' twitter-search '{term}' + "near:Los Angeles" > input/search/{term}-tweets.json

In [65]:
src = df.copy()

In [38]:
src.head()

,date,content
0,2021-02-13 23:41:45+00:00,"J’ai écrit une lettre à Macron mais je crois j’ai été super méchante, je l’envoie quand même ou il va pleurer ? (Fin plutôt le pauvre sous fifre qui va la lire)"
1,2021-02-13 23:23:28+00:00,I wanna do more fuckshit in my daily life
2,2021-02-13 23:14:07+00:00,im sorry i make my tweets unreplyable a lot of the time i just don’t want anyone saying some fuckshit underneath it dhdjh
3,2021-02-13 23:03:27+00:00,"""you seem happy"" best compliment ever."
4,2021-02-13 22:53:48+00:00,every time i see his face i know he’s about to do some fuckshit https://t.co/GVZ9tmuPOc


### Process dates

In [39]:
src["date"] = pd.to_datetime(src["date"], utc=True)
src["pacific_created_at"] = pd.to_datetime(src["date"], utc=True).dt.tz_convert(
    "America/Los_Angeles"
)

In [40]:
src["date"] = pd.to_datetime(src["pacific_created_at"]).dt.strftime("%m/%d/%Y")
src["date"] = pd.to_datetime(src["date"])
src["year"] = src["pacific_created_at"].dt.year
src["month"] = src["pacific_created_at"].dt.month
src["day"] = src["pacific_created_at"].dt.day
src["hour"] = src["pacific_created_at"].dt.hour
src["minute"] = src["pacific_created_at"].dt.minute
src["time"] = src["pacific_created_at"].dt.time

In [41]:
# src["id"] = src["id"].astype(str)
src["year"] = src["year"].astype(str)
src["month"] = src["month"].astype(str)
src["day"] = src["day"].astype(str)
src["hour"] = src["hour"].astype(str)

### Make a copy of the dataframe and ensure the text of the tweet contains 'doomscrolling'

In [44]:
df = src[src["content"].str.contains(f"{search}")].copy()

### Unpack the nested json

In [45]:
data_dict = df.to_dict("records")

In [46]:
# df["user_name"] = pd.json_normalize(df["user"])["username"]
# df["user_displayname"] = pd.json_normalize(df["user"])["displayname"]
# df["user_verified"] = pd.json_normalize(df["user"])["verified"]
# df["user_description"] = pd.json_normalize(df["user"])["description"]
# df["user_followersCount"] = pd.json_normalize(df["user"])["followersCount"]
# df["user_location"] = pd.json_normalize(df["user"])["location"]

KeyError: 'user'

### Slim down the dataframe

In [ ]:
df_slim = df[
    [
        "date",
        "content",
        "url",
        "replyCount",
        "retweetCount",
        "likeCount",
        "quoteCount",
        "user_name",
        "user_displayname",
        "user_verified",
        "user_description",
        "user_followersCount",
        "user_location",
        "month",
        "day",
        "hour",
        "minute",
        "time",
        "year",
    ]
]

In [ ]:
df_slim.year.value_counts()

---

In [ ]:
daily = df_slim.groupby(["date"]).agg("size").reset_index(name="count")

In [ ]:
daily.head()

In [ ]:
daily.sort_values("count", ascending=False).head(5)

In [ ]:
df_slim.head()

---

### Export

In [ ]:
bejesus_slim.to_csv("output/search/bejesus_slim.csv", index=False)